In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x1044defc0>

In [3]:
next(my_coro)

-> coroutine started


In [4]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [5]:
import inspect
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

In [6]:
def simple_coroutine2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [7]:
my_coro2 = simple_coroutine2(14)

In [8]:
inspect.getgeneratorstate(my_coro2)

'GEN_CREATED'

In [9]:
next(my_coro2)

-> Started: a = 14


14

In [10]:
my_coro2.send(28)

-> Received: b = 28


42

In [11]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [2]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [14]:
coro_average = averager()
next(coro_average)

In [15]:
coro_average.send(10)

10.0

In [17]:
coro_average.send(20)

15.0

In [18]:
coro_average.send(40)

23.333333333333332

In [6]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [7]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count    

In [9]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [10]:
coro_avg.send(60)

50.0

In [11]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [12]:
coro_avg.send(60)

StopIteration: 

In [13]:
coro_avg.close()

In [1]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [3]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
list(gen())

['A', 'B', 1, 2]

In [4]:
def gen():
    yield from 'AB'
    yield from range(1, 3)
    
list(gen())

['A', 'B', 1, 2]

In [13]:
# BEGIN YIELD_FROM_AVERAGER
from collections import namedtuple
from pprint import pprint

Result = namedtuple('Result', 'count average')


# the subgenerator
def averager():  # <1>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield  # <2>
        if term is None:  # <3>
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # <4>


# the delegating generator
def grouper(results, key):  # <5>
    while True:  # <6>
        results[key] = yield from averager()  # <7>


# the client code, a.k.a. the caller
def main(data):  # <8>
    results = {}
    for key, values in data.items():
        group = grouper(results, key)  # <9>
        next(group)  # <10>
        for value in values:
            group.send(value)  # <11>
        group.send(None)  # important! <12>

    pprint(results)  # uncomment to debug
#     report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}



In [14]:
main(data)

{'boys;kg': Result(count=9, average=40.422222222222224),
 'boys;m': Result(count=9, average=1.3888888888888888),
 'girls;kg': Result(count=10, average=42.040000000000006),
 'girls;m': Result(count=10, average=1.4279999999999997)}


In [16]:
results = {}
ndata = ('boys;m', [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46])
group = grouper(results, 'boys;m')
next(group)

In [17]:
type(group)

generator

In [18]:
for value in ndata[1]:
    group.send(value)

In [19]:
group.send(None)

In [20]:
results

{'boys;m': Result(count=9, average=1.3888888888888888)}

## 16.8 `yield from`의 의미
---
* 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자(즉, 클라이언트)에 바로 전달된다.
* `send()`를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 `None`이면 하위 제너레이터의 `__next__()` 메서드가 호출된다. 전달된 값이 `None`이 아니면 하위 제너레이터의 `send()` 메서드가 호출된다. 호출된 메서드에서 `StopIteration` 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
* 제너레이터나 하위 제너레이터에서 `return expr` 문을 실행하면, 제너레이터를 빠져나온 후 `StopIteration(expr)` 예외가 발생한다.
* 하위 제너레이터가 실행을 마친 후 발생한 `StopIteration`예외의 첫 번째 인수가 `yield from`표현식의 값이 된다.
* 대표 제너레이터에 던져진 `GeneratorExit` 이외의 예외는 하위 제너레이터의 `throw()` 메서드에 전달된다. `throw()` 메서드를 호출해서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
* GeneratorExit 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 `close()` 메서드가 호출되면 하위 제너레이터의 `close()` 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 GeneratorExit 예외가 발생한다.